In [67]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from itertools import combinations
from sklearn.metrics.pairwise import pairwise_distances

%matplotlib inline

In [68]:
df = pd.read_csv('/content/sample-data.csv')
print(df.shape)
df.head()

(500, 2)


,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


In [69]:
tfidf = TfidfVectorizer(
    analyzer='word',
    ngram_range = (1, 3),
    min_df = 0,
    stop_words = 'english'
)

X_tfidf = tfidf.fit_transform(df['description'])
X_tfidf

<500x52262 sparse matrix of type '<class 'numpy.float64'>'
	with 148989 stored elements in Compressed Sparse Row format>

In [70]:
def get_similar_items(X_tfidf, id, topn=5):
  query = X_tfidf[id]
  scores = X_tfidf.dot(query.T).toarray().ravel()
  best = np.argpartition(scores, -topn)[-topn:]
  return sorted(zip(best, scores[best]), key=lambda x: -x[1])


In [71]:
similar_items = get_similar_items(X_tfidf, id=1)

In [39]:
def generate_random_vectors(dim, n_vectors):
  return np.random.randn(dim, n_vectors)

 Now, We'd like to decide which bin each documents should go. Since we generated 16 random vectors, we have 16 bits to represent the bin index.
 The first bit is given by the sign of the dot product b/w the first random vector and the document's TF-IDF vector and so on.

Dimension:  (1, 16)


array([[False, False, False, False, False, False,  True,  True,  True,
        False, False,  True, False, False, False, False]])

Convert the resulting bin to integer representation for convenience.

We can use rules of binary number representation to perform the conversion(dot product b/w the document vector and the vector consisting of powers of 2).

[32768 16384  8192  4096  2048  1024   512   256   128    64    32    16
     8     4     2     1]
[912]


In [43]:
indices_bits = X_tfidf.dot(random_vectors) >= 0
print(indices_bits.shape)

bin_indices = indices_bits.dot(power_of_2)
bin_indices.shape

(500, 16)


(500,)

Given the integer bin indices for the documents, we would curate the list of document IDs that belong to each bin. Since a list is to be maintained for each unique bin index, a dictionary of lists is used.

In [73]:
def train_lsh(X_tfidf, n_vectors, seed=None):
  if seed is not None:
    np.random.seed(seed)
  
  dim = X_tfidf.shape[1]
  random_vectors = generate_random_vectors(dim, n_vectors)

  indices_bits = X_tfidf.dot(random_vectors) >= 0
  power_of_2 = 1 << np.arange(n_vectors - 1, -1, step=-1)
  bin_indices = indices_bits.dot(power_of_2)
  
  table = defaultdict(list)  #table[i] is the list of document ids with bin index equal to i

  for idx, bin_index in enumerate(bin_indices):
    table[bin_index].append(idx)

  model = {'table': table,
           'random_vectors': random_vectors,
           'bin_indices': bin_indices,
           'indices_bits': indices_bits}

  return model

In [74]:
n_vectors = 16
model = train_lsh(X_tfidf, n_vectors, seed=143)

After generating our LSH model, let's examine the generated bins to get a deeper understanding of them. Recall that during the background section, given a product's tfidf vector representation, we were able to find its similar product using standard cosine similarity. Here, we will look at these similar products' bins to see if the result matches intuition. Remember the idea behind LSH is that similar data points will tend to fall into nearby bins.

In [75]:
similar_items_ids = [i for i, _ in similar_items]
bits1 = model['indices_bits'][similar_items_ids[0]]
bits2 = model['indices_bits'][similar_items_ids[1]]

print('bits 1: ', bits2)
print('bits 2: ', bits2)

print('Number of agreed bins: ', np.sum(bits1 == bits2))

bits 1:  [ True False False False False False  True False False  True False False
  True  True  True False]
bits 2:  [ True False False False False False  True False False  True False False
  True  True  True False]
Number of agreed bins:  11


Now, we define the logic for searching nearby neighbors.

For a given query vector and trained LSH model's table return all candidate neighbors with the specified search radius

In [76]:
def search_nearby_bins(query_bin_bits, table, search_radius=3, candidate_set=None):
  if candidate_set is None:
    candidate_set = set()

  n_vectors = query_bin_bits.shape[0]
  power_of_2 = 1 << np.arange(n_vectors - 1, -1, step=-1)

  for different_bits in combinations(range(n_vectors), search_radius):
    index = list(different_bits)
    alternate_bits = query_bin_bits.copy()
    alternate_bits[index] = np.logical_not(alternate_bits[index])

    nearby_bin = alternate_bits.dot(power_of_2)

    if nearby_bin in table:
      candidate_set.update(table[nearby_bin])

  return candidate_set

Now we use above searching function for nearby bins logic to search for similar document and return a dataframe that contains the most similar data points according to LSH and their distances.

In [77]:
def get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=3):
  table = model['table']
  random_vectors = model['random_vectors']

  bin_index_bits = np.ravel(query_vector.dot(random_vectors) >= 0)

  candidate_set = set()

  for search_radius in range(max_search_radius + 1):
    candidate_set = search_nearby_bins(bin_index_bits, table, search_radius, candidate_set)
    
  candidate_list = list(candidate_set)
  candidates = X_tfidf[candidate_list]
  distance = pairwise_distances(candidates, query_vector, metric='cosine').flatten()

  distance_col = 'distance'
  nearest_neighbors = pd.DataFrame({
       'id': candidate_list, distance_col: distance
  }).sort_values(distance_col).reset_index(drop=True)

  return nearest_neighbors

In [65]:
print('Original similar items:\n' + str(similar_items))

id = 1
query_vector = X_tfidf[id]
nearest_neighbors = get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=5)
print('Dimension: ', nearest_neighbors.shape)
nearest_neighbors.head()

Original similar items:
[(1, 1.0000000000000013), (2, 0.41816639921615745), (18, 0.11546382098627585), (493, 0.11303392245400211), (299, 0.11247854521091623)]
Dimension:  (67, 2)


,id,distance
0,1,2.220446e-16
1,2,5.818336e-01
2,317,9.008780e-01
3,213,9.117783e-01
4,272,9.173818e-01


In the above result we use max_search_radius of 5, so LSH search wasn't capable of retrieving the actual most similar items to our target data point. Its  expected as LSH is an approximate nearest neighbourhood search method.

Let's increase the max_search_radius to 10 to retrieve almost all the most similar items.

In [78]:
nearest_neighbors  = get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=10)
print('Dimension: ', nearest_neighbors.shape)
nearest_neighbors.head()

Dimension:  (455, 2)


,id,distance
0,1,2.220446e-16
1,2,5.818336e-01
2,18,8.845362e-01
3,493,8.869661e-01
4,299,8.875215e-01


In [79]:
nearest_neighbors.head().merge(df, on='id', how='inner')

,id,distance,description
0,1,2.220446e-16,Active classic boxers - There's a reason why o...
1,2,5.818336e-01,Active sport boxer briefs - Skinning up Glory ...
2,18,8.845362e-01,Cap 1 bottoms - Spring skiing is as transient ...
3,493,8.869661e-01,'73 logo t-shirt - Patagonia's timeless '73 Lo...
4,299,8.875215e-01,Active boy shorts - We've worn these versatile...
